In [0]:
# bibliotecas usadas
import os
import numpy as np
import pandas as pd
from scipy import optimize
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
from mpl_toolkits.mplot3d import Axes3D, axes3d
%matplotlib inline

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [0]:
!git clone http://github.com/rodolfojbrandao/PhD
%cd PhD

fatal: destination path 'PhD' already exists and is not an empty directory.
/content/PhD


In [0]:
DAT=pd.read_csv("trabalho01redes2.csv")
print(DAT)

    pressure  temperature  co2        is
0        100           60  0.3  0.043365
1        300           50  0.2  0.068071
2        200           40  0.4  0.072746
3        200           60  0.2  0.061583
4        200           40  0.2  0.047607
5        300           60  0.3  0.100949
6        100           50  0.4  0.055266
7        100           50  0.2  0.026984
8        300           40  0.3  0.070051
9        200           60  0.4  0.133478
10       200           50  0.3  0.087583
11       100           40  0.3  0.021871
12       200           50  0.3  0.083618
13       200           50  0.3  0.071142
14       300           50  0.4  0.121988
15       300           50  0.3  0.056339
16       100           50  0.3  0.035870


In [0]:
# separa as colunas de interesse
Index=range(4)
Dat_Select=[]
Dat_Select=DAT.iloc[:,Index].copy()

In [0]:
# separa os dados em normatizados e originais
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)

In [0]:
# separa os dados em treino e teste
train=DAT.sample(frac=0.85,random_state=6) # poderia ser frac=0.5? sim ou nao?
test=DAT.drop(train.index)

In [0]:
# obtem as variaveis independentes (treino e teste)
x_train=train.iloc[:,[0,1,2]]
x_test=test.iloc[:,[0,1,2]]
X_OLD=DAT_OLD.iloc[:,[0,1,2]]

In [0]:
# obtem as variaveis dependentes (treino e teste)
Index=[3]
Y_OLD=DAT_OLD.iloc[:,Index]
y_train_Y=train.iloc[:,Index]
y_test__Y=test.iloc[:,Index]
print(Y_OLD.head())

         is
0  0.043365
1  0.068071
2  0.072746
3  0.061583
4  0.047607


In [0]:
# define a rede neural Y
clf_Y = MLPRegressor(solver='lbfgs',activation='tanh',alpha=1e-5,hidden_layer_sizes=(6), 
                    random_state=3)

In [0]:
# treina a rede neural Y
clf_Y.fit(x_train, y_train_Y)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='tanh', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=6, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=3, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [0]:
# usa a rede neural treina em calculos para treino e teste Y
y_calc_train=clf_Y.predict(x_train)
x_test=x_train
y_calc_test=clf_Y.predict(x_test)


In [0]:
# transforma a saida da rede em dataframe 
y_calc_train=pd.DataFrame(y_calc_train)
y_calc_test=pd.DataFrame(y_calc_test)
col_names=list(Y_OLD)
y_calc_train.columns = col_names
y_calc_test.columns = col_names

In [0]:
# transforma dos dados calculados na forma original (nao normatizado)
y_calc_train=Original(y_calc_train,Y_OLD)
y_calc_test=Original(y_calc_test,Y_OLD)

In [0]:
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

def D_Desirability(y,U,T,L):
    if y.all()<L:
        d=0
    if (y.all()>=L)&(y<=T):
        d=(y-L)/(T-L)
    if (y.all()>=T)&(y<+U):
        d=(U-y)/(U-T)
    if (y.all()>U):
        d=0
    return d

def Desirability(y,U,T,L):
    if y.item()<L:
        d=0
    if (y.item()>=L)&(y<=T):
        d=(y-L)/(T-L)
    if (y.item()>=T)&(y<+U):
        d=(U-y)/(U-T)
    if (y.item()>U):
        d=0
    return d

In [0]:
def function(x):
    global clf_Y
    Num=int(x.size/2)
    x=x.reshape(Num,3)
    
    df = pd.DataFrame(x)
    col_names=['x','y']
    df.columns = col_names
    Y=clf_Y.predict(df)
    
    OLD_Y=OLD[['Y']]
       
    Y=pd.DataFrame(Y)
    
    Y.columns=list(OLD_Y)
    
    Y=Original(Y,OLD_Y)
    
    Y=np.array(np.squeeze(Y))
    
    U=0.13348
    T=0.077675
    L=0.021871
    d1 =  Desirability(Y,U,T,L)

    d=(-1)*(d1)
    fun=d
    
    return fun



In [0]:
bounds = [(-2, 2),(-2, 2), (-2, 2)]
OLD=Y_OLD
ret = optimize.differential_evolution(function,bounds,
                                      init="latinhypercube", # obs: init="random"
                                      maxiter=200, popsize=100,recombination=0.5)

ValueError: ignored

In [0]:
print("global maximum: x = ",ret.x, "f(x0) = ",ret.fun)

In [0]:
# funcao retorna os dados a forma original
def Descodifica(x,x_old,strings):
    k=0
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[k]=x[k]*b+a
        k=k+1
    return x

In [0]:
strings=['pressure','temperature', 'co2']
DAT=pd.read_csv("trabalho01redes2.csv")
xcalc=[]
xcalc=Descodifica(ret.x,DAT,strings).copy()
print("global maximum: x = ",xcalc, "f(x) = ",ret.fun)

pressure -- temperature -- co2 -- is 

expe:           200 -- 60 -- 0.4 -- 0.1334

ANN :         283 -- 60 -- 0.4 -- 0.1385